# Finger 3: Graph Coloring

#### Alumno: Juan Pablo Rombolá

In [154]:
# Algoritmo de tipo Greedy
# Ahora empieza con los nodos con mayor centralidad
nombre_archivo = "gc_1000_1"

entrada = sc.textFile(nombre_archivo + ".txt", 8)
header = entrada.take(1)[0]
cantidad_vertices = int(header.split()[0])

nodo_adyacentes = (entrada.filter(lambda  line: line != header)\
                        .map(lambda line: (int(line.split()[0]), [int(line.split()[1])]))\
                        + entrada.filter(lambda  line: line != header)\
                        .map(lambda line: (int(line.split()[1]), [int(line.split()[0])])))\
                        .reduceByKey(lambda a,b: a + b)\
                        .sortBy(lambda a: -len(a[1]))\
                        .collect()

resultado = [-1] * cantidad_vertices
for par in nodo_adyacentes:
    colores_usados = sc.parallelize(par[1], 8)\
                        .map(lambda a: resultado[a])\
                        .collect()
            
    color_actual = sc.parallelize(range(0, max(colores_usados) + 2), 8)\
                    .filter(lambda a: a not in colores_usados)\
                    .takeOrdered(1, lambda a: a)
            
    resultado[par[0]] = color_actual[0]

In [156]:
# Salida
dir_salida = nombre_archivo + "_resultado.txt"
with open(dir_salida, "w") as outfile:
    outfile.write( str(max(resultado) + 1) + "\n" )
    for i in range(cantidad_vertices):
        if i == (cantidad_vertices - 1):
            outfile.write( str(resultado[i]) )
        else:
            outfile.write( str(resultado[i]) + "\n" )

In [157]:
# Verificación de color          
entrada = sc.textFile(nombre_archivo + "_resultado.txt")
header = entrada.take(1)[0]
resultado = entrada.filter(lambda  line: line != header)\
                .map(lambda line: int(line)).collect()

for par in nodo_adyacentes:
    for i in par[1]:
        if resultado[par[0]] == resultado[i]:
            print "Mal coloreado"
print "Bien coloreado"

Bien coloreado
